<a href="https://colab.research.google.com/github/GustaveRw/NLP-Fellowship/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets transformers[sentencepiece] seqeval evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 15.4 MB/s 
     |████████████████████████████████| 5.8 MB 63.6 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 81 kB 11.4 MB/s 
     |████████████████████████████████| 182 kB 77.6 MB/s 
     |████████████████████████████████| 212 kB 74.5 MB/s 
     |████████████████████████████████| 132 kB 76.5 MB/s 
     |████████████████████████████████| 127 kB 57.9 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 
     |████████████████████████████████| 1.3 MB 56.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=1e808031448c2385c8bfd4c8a0b4a39ff845c3de06433af5c208e19b398b7e0d
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
  Attempting uninstall: urllib3
    Fo

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/NLP_Fellowship_2022/Notebooks/Week 8")
!ls

data				       pt_save_pretrained
distilbert-base-uncased-finetuned-ner  Summarization.ipynb
mt_save_pretrained		       t5-small-finetuned-xsum
NER.ipynb			       translation.ipynb
ner_save_pretrained		       xlm-roberta-base-finetuned-ner
opus-mt-rw-en-finetuned-RW-to-EN


In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("masakhaner", "kin")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2116 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/302 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/605 [00:00<?, ? examples/s]

Dataset masakhaner downloaded and prepared to /root/.cache/huggingface/datasets/masakhaner/kin/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2116
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 302
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 605
    })
})

In [ ]:

dataset["train"][0]

{'id': '0',
 'tokens': ['Ambasaderi',
  'wa',
  'EU',
  'mu',
  'Rwanda',
  ',',
  'Nicola',
  'Bellomo',
  'yagize',
  'ati',
  '“',
  'Inkunga',
  'yacu',
  'ni',
  'imwe',
  'mu',
  'nkunga',
  'yagutse',
  'yiswe',
  '#',
  'TeamEurope',
  '.'],
 'ner_tags': [0,
  0,
  3,
  0,
  5,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [ ]:
dataset["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None), length=-1, id=None)

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['<s>',
 '▁Ambasad',
 'eri',
 '▁wa',
 '▁EU',
 '▁mu',
 '▁Rwanda',
 '▁',
 ',',
 '▁Nicola',
 '▁Bello',
 'mo',
 '▁ya',
 'gi',
 'ze',
 '▁ati',
 '▁“',
 '▁In',
 'kung',
 'a',
 '▁ya',
 'cu',
 '▁ni',
 '▁im',
 'we',
 '▁mu',
 '▁nk',
 'unga',
 '▁ya',
 'gut',
 'se',
 '▁y',
 'is',
 'we',
 '▁#',
 '▁Team',
 'Europe',
 '▁',
 '.',
 '</s>']

In [ ]:
# A single word corresponding to a single label may now be split into two subwords. You’ll need to realign the tokens and labels by:

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_ner = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of sha

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [ ]:
id2label = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-DATE",
    8: "I-DATE",
}
label2id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-DATE": 7,
    "I-DATE": 8,
}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=9, id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-ner",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ner["train"],
    eval_dataset=tokenized_ner["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens, id. If ner_tags, tokens, id are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2116
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 665
  Number of trainable parameters = 277459977
You're using a XLMRobertaTokenizerFast tokenizer. Please note tha

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.326596,0.319030,0.353306,0.335294,0.901592
2,No log,0.224158,0.498450,0.664256,0.569531,0.934066
3,No log,0.180488,0.571188,0.700413,0.629234,0.943857
4,0.287600,0.169883,0.631676,0.708678,0.667965,0.948577
5,0.287600,0.173881,0.614437,0.721074,0.663498,0.948014


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens, id. If ner_tags, tokens, id are not expected by `XLMRobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 605
  Batch size = 16
Saving model checkpoint to xlm-roberta-base-finetuned-ner/checkpoint-133
Configuration saved in xlm-roberta-base-finetuned-ner/checkpoint-133/config.json
Model weights saved in xlm-roberta-base-finetuned-ner/checkpoint-133/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-ner/checkpoint-133/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-ner/checkpoint-133/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens, id. If ner_tags, tokens, id 

TrainOutput(global_step=665, training_loss=0.2402518638094565, metrics={'train_runtime': 341.2453, 'train_samples_per_second': 31.004, 'train_steps_per_second': 1.949, 'total_flos': 604848406062600.0, 'train_loss': 0.2402518638094565, 'epoch': 5.0})

In [ ]:
pt_save_directory = "./ner_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

tokenizer config file saved in ./ner_save_pretrained/tokenizer_config.json
Special tokens file saved in ./ner_save_pretrained/special_tokens_map.json
Configuration saved in ./ner_save_pretrained/config.json
Model weights saved in ./ner_save_pretrained/pytorch_model.bin


In [ ]:
pt_model = AutoModelForTokenClassification.from_pretrained("./ner_save_pretrained")
tokenizer = AutoTokenizer.from_pretrained("./ner_save_pretrained")

loading configuration file ./ner_save_pretrained/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./ner_save_pretrained",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": 7,
    "B-LOC": 5,
    "B-ORG": 3,
    "B-PER": 1,
    "I-DATE": 8,
    "I-LOC": 6,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embeddi

In [ ]:
from transformers import pipeline
ner = pipeline(task="ner", model=pt_model, tokenizer=tokenizer)

In [ ]:
ner("Umuherwe Elon Musk kuri uyu wa Kabiri yatangaje ko bidatinze azegura ku mwanya w’Umuyobozi Mukuru wa Twitter igihe cyose azabona umusimbura.")

[{'entity': 'B-PER',
  'score': 0.9031747,
  'index': 5,
  'word': '▁El',
  'start': 9,
  'end': 11},
 {'entity': 'I-PER',
  'score': 0.4170966,
  'index': 6,
  'word': 'on',
  'start': 11,
  'end': 13},
 {'entity': 'I-PER',
  'score': 0.9036782,
  'index': 7,
  'word': '▁Musk',
  'start': 14,
  'end': 18},
 {'entity': 'B-DATE',
  'score': 0.64781046,
  'index': 9,
  'word': '▁',
  'start': 24,
  'end': 25},
 {'entity': 'I-DATE',
  'score': 0.5136269,
  'index': 10,
  'word': 'uyu',
  'start': 24,
  'end': 27},
 {'entity': 'I-DATE',
  'score': 0.89714223,
  'index': 11,
  'word': '▁wa',
  'start': 28,
  'end': 30},
 {'entity': 'I-DATE',
  'score': 0.9325558,
  'index': 12,
  'word': '▁Ka',
  'start': 31,
  'end': 33},
 {'entity': 'I-DATE',
  'score': 0.9063865,
  'index': 13,
  'word': 'biri',
  'start': 33,
  'end': 37},
 {'entity': 'B-ORG',
  'score': 0.91194797,
  'index': 38,
  'word': '▁Twitter',
  'start': 101,
  'end': 108}]